In [1]:
!pip install keras-tuner --upgrade

     |████████████████████████████████| 97 kB 3.5 MB/s 


In [11]:
import keras_tuner as kt
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.optimizers import Adam

In [3]:
import numpy as np

In [4]:
(train_images,train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [5]:
train_images = train_images/255.0
test_images = test_images/255.0

In [6]:
train_images[0].shape

(28, 28)

In [7]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [8]:
train_images.shape

(60000, 28, 28, 1)

In [10]:
def build_model(hp):  
  model = Sequential([
        Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
        Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
        Flatten(),
        Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
        Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [12]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [13]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 44s]
val_accuracy: 0.9139999747276306

Best val_accuracy So Far: 0.9139999747276306
Total elapsed time: 00h 06m 16s
INFO:tensorflow:Oracle triggered exit


In [14]:
model=tuner_search.get_best_models(num_models=1)[0]

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 64)        1664      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 48)        76848     
_________________________________________________________________
flatten (Flatten)            (None, 19200)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                1843296   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 1,922,778
Trainable params: 1,922,778
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.1668 - accuracy: 0.9371 - val_loss: 0.2687 - val_accuracy: 0.9133
Epoch 5/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.1340 - accuracy: 0.9503 - val_loss: 0.2821 - val_accuracy: 0.9047
Epoch 6/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.1037 - accuracy: 0.9613 - val_loss: 0.3223 - val_accuracy: 0.9073
Epoch 7/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.0783 - accuracy: 0.9704 - val_loss: 0.4039 - val_accuracy: 0.9023
Epoch 8/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.0594 - accuracy: 0.9781 - val_loss: 0.4036 - val_accuracy: 0.9083
Epoch 9/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.0502 - accuracy: 0.9809 - val_loss: 0.4441 - val_accuracy: 0.9075
Epoch 10/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.0409 - accuracy: 0.9849 - val_loss: 0.4560 - val_a